### Try-it 8.1: The "Best" Model

This module was all about regression and using Python's scikitlearn library to build regression models.  Below, a dataset related to real estate prices in California is given. While many of the assignments you have built and evaluated different models, it is important to spend some time interpreting the resulting "best" model.  


Your goal is to build a regression model to predict the price of a house in California.  After doing so, you are to *interpret* the model.  There are many strategies for doing so, including some built in methods from scikitlearn.  One example is `permutation_importance`.  Permutation feature importance is a strategy for inspecting a model and its features importance.  

Take a look at the user guide for `permutation_importance` [here](https://scikit-learn.org/stable/modules/permutation_importance.html).  Use  the `sklearn.inspection` modules implementation of `permutation_importance` to investigate the importance of different features to your regression models.  Share these results on the discussion board.

In [ ]:
# %pip install interpret

In [ ]:
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cali = pd.read_csv('data/housing.csv')

In [ ]:
cali.head()

In [ ]:
cali.info()

## Qucik EDA

Check missing values

In [ ]:
cali.isnull().sum()

`total_bedrooms` has 207 missing values (1% of data)

In [ ]:
cali["total_bedrooms"].plot.box()

Since the missing values are around 1%, drop rows that has missing rows

In [ ]:
print(f"Shape before drop NaN {cali.shape}")
cali = cali.dropna()
print(f"After before drop NaN {cali.shape}")

### Closer to ocean greater house value

> Order of the ocean_proximity category will follows the order of mean of house value by the proximity group

In [ ]:
# print(cali["ocean_proximity"].value_counts().index.to_list())
ocean_proximity_category = [cali.groupby(["ocean_proximity"]).mean()["median_house_value"].sort_values(ascending=True).index.to_list()]
cali.groupby(["ocean_proximity"]).mean()["median_house_value"].sort_values(ascending=True)

In [ ]:
print(ocean_proximity_category)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
cali.corr()[["median_house_value"]].nlargest(columns="median_house_value", n = 5)

`median_income` has the hightest correlation wtih `median_house_vale`

In [ ]:
samples = cali.sample(n = 1000)
plt.scatter(cali["median_income"], cali["median_house_value"])

## get base line model

Use `Lineare Regression' model to predict 

Use numeric features, exclude categorical features

In [ ]:
best_model = ''
best_mse = 0

In [ ]:
labelName = "median_house_value"
features = cali.drop([labelName], axis=1)
numericFeatures = features.select_dtypes(include="number")
categoricalFeatures = features.select_dtypes(include="object")

In [ ]:
print("numeric features: ", numericFeatures.columns)
print("categorical featuer: ", categoricalFeatures.columns)
print("label: ", labelName)

In [ ]:
X_base = cali[numericFeatures.columns]
y_base = cali[labelName]
X_train, X_test, y_train, y_test = train_test_split(X_base, y_base, test_size=0.3, random_state=1234)

In [ ]:
base_model = Pipeline([
    ("lr", LinearRegression())
]).fit(X_base, y_base)

mse_base = mean_squared_error(y_test, base_model.predict(X_test))
mae_base = mean_absolute_error(y_test, base_model.predict(X_test))

print(f"Base model metrics: \nmse {mse_base:.2f},\nmae {mae_base:.2f}")

best_model = base_model
best_mse = mse_base

## Apply feature engineering 

Use PolynomialFeatures

## Select the best model, fine tune model 2


`degree = 3` is the best for this case

In [ ]:
mses_base = []
maes_base = []

for i in range(1, 6):
    model_2 = Pipeline([
        ("pf", PolynomialFeatures(degree=i)),
        ("lr", LinearRegression())
    ])

    model_2.fit(X_train, y_train)
    mses_base.append(mean_squared_error(y_test, model_2.predict(X_test)))
    maes_base.append(mean_absolute_error(y_test, model_2.predict(X_test)))

plt.plot([i for i in range(1, 5)], mses_base[:4], "go--")

In [ ]:
minMse = np.min(mses_base)
optDegree = mses_base.index(minMse) + 1
print(f"The optimal degree for Polynomial Features is {optDegree}")

model_2 = Pipeline([
    ("pf", PolynomialFeatures(degree=optDegree)),
    ("lr", LinearRegression())
])

model_2.fit(X_train, y_train)
mse_model2 = mean_squared_error(y_test, model_2.predict(X_test))

In [ ]:
print(f"MSE: {mse_model2}")
if(best_mse > mse_model2):
    print("Model_2 is better than Model_Base")
    best_model = model_2
    best_mse = mse_model2
else:
    print("Model_Base is better")

## Select the best model, fine tune model 3

In [ ]:
X = cali.drop([labelName], axis=1)
y = cali[labelName]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [ ]:
X_train.head()

In [ ]:
print(numericFeatures.columns)

print()

print(ocean_proximity_category)

In [ ]:
transformFeatures = make_column_transformer(
    (PolynomialFeatures(degree=3), numericFeatures.columns),
    (OrdinalEncoder(categories=ocean_proximity_category), ["ocean_proximity"])
)

model_3 = Pipeline([
    ("transform", transformFeatures),
    ("lr", LinearRegression())
])

print(model_3)

model_3.fit(X_train, y_train)

mse_model_3 = mean_squared_error(y_test, model_3.predict(X_test))

print(f"MSE: {mse_model_3}")
if(best_mse > mse_model_3):
    print("Model_3 is better than Model_Base")
    best_model = model_3
    best_mse = mse_model_3
else:
    print("No imporvement")
# print(mean_absolute_error(y_test, model_3.predict(X_test)))


## Select the best model, fine tune model 4

In [ ]:
transformFeatures = make_column_transformer(
    (OrdinalEncoder(categories=ocean_proximity_category), ["ocean_proximity"])
)

model_4 = Pipeline([
    ("transform", transformFeatures),
    ("lr", LinearRegression())
])

print(model_4)

model_4.fit(X_train, y_train)

mse_model_4 = mean_squared_error(y_test, model_4.predict(X_test))

print(f"MSE: {mse_model_4}")

if(best_mse > mse_model_4):
    print("Model_4 is better than Model_Base")
    best_model = model_4
    best_mse = mse_model_4
else:
    print("No imporvement")
# print(mean_absolute_error(y_test, model_3.predict(X_test)))


In [ ]:
print(best_model)
print(best_mse)

## interpret the best model

In [ ]:
X_base = cali[numericFeatures.columns]
y_base = cali[labelName]
X_train, X_test, y_train, y_test = train_test_split(X_base, y_base, test_size=0.3, random_state=1234)


In [ ]:
from sklearn.inspection import permutation_importance
r = permutation_importance(best_model, X_test, y_test,
                           n_repeats=30,
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{X_test.columns[i]:<8}\t\t"
              f"{r.importances_mean[i]:.3f}\t\t"
              f" +/- {r.importances_std[i]:.3f}")

## Use Interpret ML

In [ ]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

In [ ]:
# model_4["transform"].fit_transform(X_train)
X_train_transform = pd.DataFrame(model_2["pf"].transform(X_train), columns=model_2["pf"].get_feature_names_out())
X_test_transform = pd.DataFrame(model_2["pf"].transform(X_test), columns=model_2["pf"].get_feature_names_out())


In [ ]:
# X_train_transform.info()

In [ ]:
from interpret.glassbox import LinearRegression
from interpret import show

lr = LinearRegression()
lr.fit(X_train_transform, y_train)

lr_global = lr.explain_global()
show(lr_global)

lr_local = lr.explain_local(X_test_transform[:5], y_train[:5])
show(lr_local)

## Conclusion

### The best model is:

- LinearRegression
- Preprosessing on numerical features using PolynominalFeature function
- No categorical feature was used

```text
Pipeline(steps=[('pf', PolynomialFeatures(degree=3)),
                ('lr', LinearRegression())])

MSE: 3733457854.427122
```

### The most important features are:
- households
- total_bestrooms
- total_rooms
- population
- latitude
- longitude
- median_income